In [1]:
import nltk
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
df = pd.read_csv('../assets/data/stock-news-dataset.csv', encoding = "ISO-8859-1")
df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2000-01-03,0,A 'hindrance to operations': extracts from the...,Scorecard,Hughes' instant hit buoys Blues,Jack gets his skates on at ice-cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,...,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl's successor drawn into scandal,The difference between men and women,"Sara Denver, nurse turned solicitor",Diana's landmine crusade put Tories in a panic,Yeltsin's resignation caught opposition flat-f...,Russian roulette,Sold out,Recovering a title
1,2000-01-04,0,Scorecard,The best lake scene,Leader: German sleaze inquiry,"Cheerio, boyo",The main recommendations,Has Cubie killed fees?,Has Cubie killed fees?,Has Cubie killed fees?,...,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man's extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn't know without the ...,Millennium bug fails to bite
2,2000-01-05,0,Coventry caught on counter by Flo,United's rivals on the road to Rio,Thatcher issues defence before trial by video,Police help Smith lay down the law at Everton,Tale of Trautmann bears two more retellings,England on the rack,Pakistan retaliate with call for video of Walsh,Cullinan continues his Cape monopoly,...,South Melbourne (Australia),Necaxa (Mexico),Real Madrid (Spain),Raja Casablanca (Morocco),Corinthians (Brazil),Tony's pet project,Al Nassr (Saudi Arabia),Ideal Holmes show,Pinochet leaves hospital after tests,Useful links
3,2000-01-06,1,Pilgrim knows how to progress,Thatcher facing ban,McIlroy calls for Irish fighting spirit,Leicester bin stadium blueprint,United braced for Mexican wave,"Auntie back in fashion, even if the dress look...",Shoaib appeal goes to the top,Hussain hurt by 'shambles' but lays blame on e...,...,Putin admits Yeltsin quit to give him a head s...,BBC worst hit as digital TV begins to bite,How much can you pay for...,Christmas glitches,"Upending a table, Chopping a line and Scoring ...","Scientific evidence 'unreliable', defence claims",Fusco wins judicial review in extradition case,Rebels thwart Russian advance,Blair orders shake-up of failing NHS,Lessons of law's hard heart
4,2000-01-07,1,Hitches and Horlocks,Beckham off but United survive,Breast cancer screening,Alan Parker,Guardian readers: are you all whingers?,Hollywood Beyond,Ashes and diamonds,Whingers - a formidable minority,...,Most everywhere: UDIs,Most wanted: Chloe lunettes,Return of the cane 'completely off the agenda',From Sleepy Hollow to Greeneland,Blunkett outlines vision for over 11s,"Embattled Dobson attacks 'play now, pay later'...",Doom and the Dome,What is the north-south divide?,Aitken released from jail,Gone aloft


In [6]:
# Lowercasing all text
for col in df.columns:
  try:
    df[col] = df[col].str.lower()
  except:
    print("not possible")  
df.head()

not possible


,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2000-01-03,0,a 'hindrance to operations': extracts from the...,scorecard,hughes' instant hit buoys blues,jack gets his skates on at ice-cold alex,chaos as maracana builds up for united,depleted leicester prevail as elliott spoils e...,hungry spurs sense rich pickings,gunners so wide of an easy target,...,flintoff injury piles on woe for england,hunters threaten jospin with new battle of the...,kohl's successor drawn into scandal,the difference between men and women,"sara denver, nurse turned solicitor",diana's landmine crusade put tories in a panic,yeltsin's resignation caught opposition flat-f...,russian roulette,sold out,recovering a title
1,2000-01-04,0,scorecard,the best lake scene,leader: german sleaze inquiry,"cheerio, boyo",the main recommendations,has cubie killed fees?,has cubie killed fees?,has cubie killed fees?,...,on the critical list,the timing of their lives,dear doctor,irish court halts ira man's extradition to nor...,burundi peace initiative fades after rebels re...,pe points the way forward to the ecb,campaigners keep up pressure on nazi war crime...,jane ratcliffe,yet more things you wouldn't know without the ...,millennium bug fails to bite
2,2000-01-05,0,coventry caught on counter by flo,united's rivals on the road to rio,thatcher issues defence before trial by video,police help smith lay down the law at everton,tale of trautmann bears two more retellings,england on the rack,pakistan retaliate with call for video of walsh,cullinan continues his cape monopoly,...,south melbourne (australia),necaxa (mexico),real madrid (spain),raja casablanca (morocco),corinthians (brazil),tony's pet project,al nassr (saudi arabia),ideal holmes show,pinochet leaves hospital after tests,useful links
3,2000-01-06,1,pilgrim knows how to progress,thatcher facing ban,mcilroy calls for irish fighting spirit,leicester bin stadium blueprint,united braced for mexican wave,"auntie back in fashion, even if the dress look...",shoaib appeal goes to the top,hussain hurt by 'shambles' but lays blame on e...,...,putin admits yeltsin quit to give him a head s...,bbc worst hit as digital tv begins to bite,how much can you pay for...,christmas glitches,"upending a table, chopping a line and scoring ...","scientific evidence 'unreliable', defence claims",fusco wins judicial review in extradition case,rebels thwart russian advance,blair orders shake-up of failing nhs,lessons of law's hard heart
4,2000-01-07,1,hitches and horlocks,beckham off but united survive,breast cancer screening,alan parker,guardian readers: are you all whingers?,hollywood beyond,ashes and diamonds,whingers - a formidable minority,...,most everywhere: udis,most wanted: chloe lunettes,return of the cane 'completely off the agenda',from sleepy hollow to greeneland,blunkett outlines vision for over 11s,"embattled dobson attacks 'play now, pay later'...",doom and the dome,what is the north-south divide?,aitken released from jail,gone aloft


In [7]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /home/volt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/volt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/volt/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/volt/nltk_data...


True

In [8]:
df = df.dropna()

In [ ]:
# Tokenizing the texts
# stop_words = set(stopwords.words('english'))

for col in df.columns:
  try:
    df[col]=df[col].apply(word_tokenize)
  except:
    print("not possible") 

df.head()

not possible


,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,[2000-01-03],0,"[a, 'hindrance, to, operations, ', :, extracts...",[scorecard],"[hughes, ', instant, hit, buoys, blues]","[jack, gets, his, skates, on, at, ice-cold, alex]","[chaos, as, maracana, builds, up, for, united]","[depleted, leicester, prevail, as, elliott, sp...","[hungry, spurs, sense, rich, pickings]","[gunners, so, wide, of, an, easy, target]",...,"[flintoff, injury, piles, on, woe, for, england]","[hunters, threaten, jospin, with, new, battle,...","[kohl, 's, successor, drawn, into, scandal]","[the, difference, between, men, and, women]","[sara, denver, ,, nurse, turned, solicitor]","[diana, 's, landmine, crusade, put, tories, in...","[yeltsin, 's, resignation, caught, opposition,...","[russian, roulette]","[sold, out]","[recovering, a, title]"
1,[2000-01-04],0,[scorecard],"[the, best, lake, scene]","[leader, :, german, sleaze, inquiry]","[cheerio, ,, boyo]","[the, main, recommendations]","[has, cubie, killed, fees, ?]","[has, cubie, killed, fees, ?]","[has, cubie, killed, fees, ?]",...,"[on, the, critical, list]","[the, timing, of, their, lives]","[dear, doctor]","[irish, court, halts, ira, man, 's, extraditio...","[burundi, peace, initiative, fades, after, reb...","[pe, points, the, way, forward, to, the, ecb]","[campaigners, keep, up, pressure, on, nazi, wa...","[jane, ratcliffe]","[yet, more, things, you, would, n't, know, wit...","[millennium, bug, fails, to, bite]"
2,[2000-01-05],0,"[coventry, caught, on, counter, by, flo]","[united, 's, rivals, on, the, road, to, rio]","[thatcher, issues, defence, before, trial, by,...","[police, help, smith, lay, down, the, law, at,...","[tale, of, trautmann, bears, two, more, retell...","[england, on, the, rack]","[pakistan, retaliate, with, call, for, video, ...","[cullinan, continues, his, cape, monopoly]",...,"[south, melbourne, (, australia, )]","[necaxa, (, mexico, )]","[real, madrid, (, spain, )]","[raja, casablanca, (, morocco, )]","[corinthians, (, brazil, )]","[tony, 's, pet, project]","[al, nassr, (, saudi, arabia, )]","[ideal, holmes, show]","[pinochet, leaves, hospital, after, tests]","[useful, links]"
3,[2000-01-06],1,"[pilgrim, knows, how, to, progress]","[thatcher, facing, ban]","[mcilroy, calls, for, irish, fighting, spirit]","[leicester, bin, stadium, blueprint]","[united, braced, for, mexican, wave]","[auntie, back, in, fashion, ,, even, if, the, ...","[shoaib, appeal, goes, to, the, top]","[hussain, hurt, by, 'shambles, ', but, lays, b...",...,"[putin, admits, yeltsin, quit, to, give, him, ...","[bbc, worst, hit, as, digital, tv, begins, to,...","[how, much, can, you, pay, for, ...]","[christmas, glitches]","[upending, a, table, ,, chopping, a, line, and...","[scientific, evidence, 'unreliable, ', ,, defe...","[fusco, wins, judicial, review, in, extraditio...","[rebels, thwart, russian, advance]","[blair, orders, shake-up, of, failing, nhs]","[lessons, of, law, 's, hard, heart]"
4,[2000-01-07],1,"[hitches, and, horlocks]","[beckham, off, but, united, survive]","[breast, cancer, screening]","[alan, parker]","[guardian, readers, :, are, you, all, whingers...","[hollywood, beyond]","[ashes, and, diamonds]","[whingers, -, a, formidable, minority]",...,"[most, everywhere, :, udis]","[most, wanted, :, chloe, lunettes]","[return, of, the, cane, 'completely, off, the,...","[from, sleepy, hollow, to, greeneland]","[blunkett, outlines, vision, for, over, 11s]","[embattled, dobson, attacks, 'play, now, ,, pa...","[doom, and, the, dome]","[what, is, the, north-south, divide, ?]","[aitken, released, from, jail]","[gone, aloft]"


In [ ]:
stop_words = set(stopwords.words('english'))
def removing_stopword(x):
  # lit=df['data'].iloc[0]
  new_lit=[]
  for i in x:
    if i in stop_words:
      continue
    else:
      new_lit.append(i)
  # print(x)
  # print(new_lit)
  return new_lit

for col in df.columns:
  try:
    df[col]=df[col].apply(removing_stopword)
  except:
    print("not possible")

not possible


In [ ]:
df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,[2000-01-03],0,"['hindrance, operations, ', :, extracts, leake...",[scorecard],"[hughes, ', instant, hit, buoys, blues]","[jack, gets, skates, ice-cold, alex]","[chaos, maracana, builds, united]","[depleted, leicester, prevail, elliott, spoils...","[hungry, spurs, sense, rich, pickings]","[gunners, wide, easy, target]",...,"[flintoff, injury, piles, woe, england]","[hunters, threaten, jospin, new, battle, somme]","[kohl, 's, successor, drawn, scandal]","[difference, men, women]","[sara, denver, ,, nurse, turned, solicitor]","[diana, 's, landmine, crusade, put, tories, pa...","[yeltsin, 's, resignation, caught, opposition,...","[russian, roulette]",[sold],"[recovering, title]"
1,[2000-01-04],0,[scorecard],"[best, lake, scene]","[leader, :, german, sleaze, inquiry]","[cheerio, ,, boyo]","[main, recommendations]","[cubie, killed, fees, ?]","[cubie, killed, fees, ?]","[cubie, killed, fees, ?]",...,"[critical, list]","[timing, lives]","[dear, doctor]","[irish, court, halts, ira, man, 's, extraditio...","[burundi, peace, initiative, fades, rebels, re...","[pe, points, way, forward, ecb]","[campaigners, keep, pressure, nazi, war, crime...","[jane, ratcliffe]","[yet, things, would, n't, know, without, movies]","[millennium, bug, fails, bite]"
2,[2000-01-05],0,"[coventry, caught, counter, flo]","[united, 's, rivals, road, rio]","[thatcher, issues, defence, trial, video]","[police, help, smith, lay, law, everton]","[tale, trautmann, bears, two, retellings]","[england, rack]","[pakistan, retaliate, call, video, walsh]","[cullinan, continues, cape, monopoly]",...,"[south, melbourne, (, australia, )]","[necaxa, (, mexico, )]","[real, madrid, (, spain, )]","[raja, casablanca, (, morocco, )]","[corinthians, (, brazil, )]","[tony, 's, pet, project]","[al, nassr, (, saudi, arabia, )]","[ideal, holmes, show]","[pinochet, leaves, hospital, tests]","[useful, links]"
3,[2000-01-06],1,"[pilgrim, knows, progress]","[thatcher, facing, ban]","[mcilroy, calls, irish, fighting, spirit]","[leicester, bin, stadium, blueprint]","[united, braced, mexican, wave]","[auntie, back, fashion, ,, even, dress, looks,...","[shoaib, appeal, goes, top]","[hussain, hurt, 'shambles, ', lays, blame, ear...",...,"[putin, admits, yeltsin, quit, give, head, start]","[bbc, worst, hit, digital, tv, begins, bite]","[much, pay, ...]","[christmas, glitches]","[upending, table, ,, chopping, line, scoring, ...","[scientific, evidence, 'unreliable, ', ,, defe...","[fusco, wins, judicial, review, extradition, c...","[rebels, thwart, russian, advance]","[blair, orders, shake-up, failing, nhs]","[lessons, law, 's, hard, heart]"
4,[2000-01-07],1,"[hitches, horlocks]","[beckham, united, survive]","[breast, cancer, screening]","[alan, parker]","[guardian, readers, :, whingers, ?]","[hollywood, beyond]","[ashes, diamonds]","[whingers, -, formidable, minority]",...,"[everywhere, :, udis]","[wanted, :, chloe, lunettes]","[return, cane, 'completely, agenda, ']","[sleepy, hollow, greeneland]","[blunkett, outlines, vision, 11s]","[embattled, dobson, attacks, 'play, ,, pay, la...","[doom, dome]","[north-south, divide, ?]","[aitken, released, jail]","[gone, aloft]"


In [ ]:
ps= PorterStemmer()
def stemming(x):
  new_lit=[]
  for i in x:
    new_lit.append(ps.stem(i))
  return new_lit

for col in df.columns:
  try:
    df[col]=df[col].apply(stemming)
  except:
    print("not possible")


not possible


In [ ]:
df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,[2000-01-03],0,"['hindranc, oper, ', :, extract, leak, report]",[scorecard],"[hugh, ', instant, hit, buoy, blue]","[jack, get, skate, ice-cold, alex]","[chao, maracana, build, unit]","[deplet, leicest, prevail, elliott, spoil, eve...","[hungri, spur, sens, rich, pick]","[gunner, wide, easi, target]",...,"[flintoff, injuri, pile, woe, england]","[hunter, threaten, jospin, new, battl, somm]","[kohl, 's, successor, drawn, scandal]","[differ, men, women]","[sara, denver, ,, nurs, turn, solicitor]","[diana, 's, landmin, crusad, put, tori, panic]","[yeltsin, 's, resign, caught, opposit, flat-foot]","[russian, roulett]",[sold],"[recov, titl]"
1,[2000-01-04],0,[scorecard],"[best, lake, scene]","[leader, :, german, sleaz, inquiri]","[cheerio, ,, boyo]","[main, recommend]","[cubi, kill, fee, ?]","[cubi, kill, fee, ?]","[cubi, kill, fee, ?]",...,"[critic, list]","[time, live]","[dear, doctor]","[irish, court, halt, ira, man, 's, extradit, n...","[burundi, peac, initi, fade, rebel, reject, ma...","[pe, point, way, forward, ecb]","[campaign, keep, pressur, nazi, war, crime, su...","[jane, ratcliff]","[yet, thing, would, n't, know, without, movi]","[millennium, bug, fail, bite]"
2,[2000-01-05],0,"[coventri, caught, counter, flo]","[unit, 's, rival, road, rio]","[thatcher, issu, defenc, trial, video]","[polic, help, smith, lay, law, everton]","[tale, trautmann, bear, two, retel]","[england, rack]","[pakistan, retali, call, video, walsh]","[cullinan, continu, cape, monopoli]",...,"[south, melbourn, (, australia, )]","[necaxa, (, mexico, )]","[real, madrid, (, spain, )]","[raja, casablanca, (, morocco, )]","[corinthian, (, brazil, )]","[toni, 's, pet, project]","[al, nassr, (, saudi, arabia, )]","[ideal, holm, show]","[pinochet, leav, hospit, test]","[use, link]"
3,[2000-01-06],1,"[pilgrim, know, progress]","[thatcher, face, ban]","[mcilroy, call, irish, fight, spirit]","[leicest, bin, stadium, blueprint]","[unit, brace, mexican, wave]","[aunti, back, fashion, ,, even, dress, look, b...","[shoaib, appeal, goe, top]","[hussain, hurt, 'shambl, ', lay, blame, earlie...",...,"[putin, admit, yeltsin, quit, give, head, start]","[bbc, worst, hit, digit, tv, begin, bite]","[much, pay, ...]","[christma, glitch]","[upend, tabl, ,, chop, line, score, goal]","[scientif, evid, 'unreli, ', ,, defenc, claim]","[fusco, win, judici, review, extradit, case]","[rebel, thwart, russian, advanc]","[blair, order, shake-up, fail, nh]","[lesson, law, 's, hard, heart]"
4,[2000-01-07],1,"[hitch, horlock]","[beckham, unit, surviv]","[breast, cancer, screen]","[alan, parker]","[guardian, reader, :, whinger, ?]","[hollywood, beyond]","[ash, diamond]","[whinger, -, formid, minor]",...,"[everywher, :, udi]","[want, :, chloe, lunett]","[return, cane, 'complet, agenda, ']","[sleepi, hollow, greeneland]","[blunkett, outlin, vision, 11]","[embattl, dobson, attack, 'play, ,, pay, later...","[doom, dome]","[north-south, divid, ?]","[aitken, releas, jail]","[gone, aloft]"


In [ ]:
import nltk
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
def lemmatize(x):
  new_lit=[]
  for i in x:
    new_lit.append(lemmatizer.lemmatize(i))
  return new_lit

for col in df.columns:
  try:
    df[col]=df[col].apply(lemmatize)
  except:
    print("not possible")


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible


In [ ]:
df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,[2000-01-03],0,"['hindranc, oper, ', :, extract, leak, report]",[scorecard],"[hugh, ', instant, hit, buoy, blue]","[jack, get, skate, ice-cold, alex]","[chao, maracana, build, unit]","[deplet, leicest, prevail, elliott, spoil, eve...","[hungri, spur, sens, rich, pick]","[gunner, wide, easi, target]",...,"[flintoff, injuri, pile, woe, england]","[hunter, threaten, jospin, new, battl, somm]","[kohl, 's, successor, drawn, scandal]","[differ, men, women]","[sara, denver, ,, nurs, turn, solicitor]","[diana, 's, landmin, crusad, put, tori, panic]","[yeltsin, 's, resign, caught, opposit, flat-foot]","[russian, roulett]",[sold],"[recov, titl]"
1,[2000-01-04],0,[scorecard],"[best, lake, scene]","[leader, :, german, sleaz, inquiri]","[cheerio, ,, boyo]","[main, recommend]","[cubi, kill, fee, ?]","[cubi, kill, fee, ?]","[cubi, kill, fee, ?]",...,"[critic, list]","[time, live]","[dear, doctor]","[irish, court, halt, ira, man, 's, extradit, n...","[burundi, peac, initi, fade, rebel, reject, ma...","[pe, point, way, forward, ecb]","[campaign, keep, pressur, nazi, war, crime, su...","[jane, ratcliff]","[yet, thing, would, n't, know, without, movi]","[millennium, bug, fail, bite]"
2,[2000-01-05],0,"[coventri, caught, counter, flo]","[unit, 's, rival, road, rio]","[thatcher, issu, defenc, trial, video]","[polic, help, smith, lay, law, everton]","[tale, trautmann, bear, two, retel]","[england, rack]","[pakistan, retali, call, video, walsh]","[cullinan, continu, cape, monopoli]",...,"[south, melbourn, (, australia, )]","[necaxa, (, mexico, )]","[real, madrid, (, spain, )]","[raja, casablanca, (, morocco, )]","[corinthian, (, brazil, )]","[toni, 's, pet, project]","[al, nassr, (, saudi, arabia, )]","[ideal, holm, show]","[pinochet, leav, hospit, test]","[use, link]"
3,[2000-01-06],1,"[pilgrim, know, progress]","[thatcher, face, ban]","[mcilroy, call, irish, fight, spirit]","[leicest, bin, stadium, blueprint]","[unit, brace, mexican, wave]","[aunti, back, fashion, ,, even, dress, look, b...","[shoaib, appeal, goe, top]","[hussain, hurt, 'shambl, ', lay, blame, earlie...",...,"[putin, admit, yeltsin, quit, give, head, start]","[bbc, worst, hit, digit, tv, begin, bite]","[much, pay, ...]","[christma, glitch]","[upend, tabl, ,, chop, line, score, goal]","[scientif, evid, 'unreli, ', ,, defenc, claim]","[fusco, win, judici, review, extradit, case]","[rebel, thwart, russian, advanc]","[blair, order, shake-up, fail, nh]","[lesson, law, 's, hard, heart]"
4,[2000-01-07],1,"[hitch, horlock]","[beckham, unit, surviv]","[breast, cancer, screen]","[alan, parker]","[guardian, reader, :, whinger, ?]","[hollywood, beyond]","[ash, diamond]","[whinger, -, formid, minor]",...,"[everywher, :, udi]","[want, :, chloe, lunett]","[return, cane, 'complet, agenda, ']","[sleepi, hollow, greeneland]","[blunkett, outlin, vision, 11]","[embattl, dobson, attack, 'play, ,, pay, later...","[doom, dome]","[north-south, divid, ?]","[aitken, releas, jail]","[gone, aloft]"


In [ ]:
def punc_removal(x):
  # using list comprehension
  listToStr = ' '.join([str(elem) for elem in x])
  non_url=re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", listToStr)
  non_url = non_url.strip()

  # hastag removal
  non_hashtag=' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",non_url).split())

  # punctuation removal
  non_punctuation = non_hashtag.translate(str.maketrans('', '', string.punctuation))

  # new line and /r removal
  no_newline = non_punctuation.replace('\n', ' ').replace('\r', '')
  return no_newline

for col in df.columns:
  try:
    df[col]=df[col].apply(punc_removal)
  except:
    print("not possible")


not possible


In [ ]:
df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2000 01 03,0,hindranc oper extract leak report,scorecard,hugh instant hit buoy blue,jack get skate ice cold alex,chao maracana build unit,deplet leicest prevail elliott spoil everton s...,hungri spur sens rich pick,gunner wide easi target,...,flintoff injuri pile woe england,hunter threaten jospin new battl somm,kohl s successor drawn scandal,differ men women,sara denver nurs turn solicitor,diana s landmin crusad put tori panic,yeltsin s resign caught opposit flat foot,russian roulett,sold,recov titl
1,2000 01 04,0,scorecard,best lake scene,leader german sleaz inquiri,cheerio boyo,main recommend,cubi kill fee,cubi kill fee,cubi kill fee,...,critic list,time live,dear doctor,irish court halt ira man s extradit northern i...,burundi peac initi fade rebel reject mandela m...,pe point way forward ecb,campaign keep pressur nazi war crime suspect,jane ratcliff,yet thing would n t know without movi,millennium bug fail bite
2,2000 01 05,0,coventri caught counter flo,unit s rival road rio,thatcher issu defenc trial video,polic help smith lay law everton,tale trautmann bear two retel,england rack,pakistan retali call video walsh,cullinan continu cape monopoli,...,south melbourn australia,necaxa mexico,real madrid spain,raja casablanca morocco,corinthian brazil,toni s pet project,al nassr saudi arabia,ideal holm show,pinochet leav hospit test,use link
3,2000 01 06,1,pilgrim know progress,thatcher face ban,mcilroy call irish fight spirit,leicest bin stadium blueprint,unit brace mexican wave,aunti back fashion even dress look bit tatti,shoaib appeal goe top,hussain hurt shambl lay blame earlier damag,...,putin admit yeltsin quit give head start,bbc worst hit digit tv begin bite,much pay,christma glitch,upend tabl chop line score goal,scientif evid unreli defenc claim,fusco win judici review extradit case,rebel thwart russian advanc,blair order shake up fail nh,lesson law s hard heart
4,2000 01 07,1,hitch horlock,beckham unit surviv,breast cancer screen,alan parker,guardian reader whinger,hollywood beyond,ash diamond,whinger formid minor,...,everywher udi,want chloe lunett,return cane complet agenda,sleepi hollow greeneland,blunkett outlin vision 11,embattl dobson attack play pay later livingston,doom dome,north south divid,aitken releas jail,gone aloft


In [ ]:
df.columns

Index(['Date', 'Label', 'Top1', 'Top2', 'Top3', 'Top4', 'Top5', 'Top6', 'Top7',
       'Top8', 'Top9', 'Top10', 'Top11', 'Top12', 'Top13', 'Top14', 'Top15',
       'Top16', 'Top17', 'Top18', 'Top19', 'Top20', 'Top21', 'Top22', 'Top23',
       'Top24', 'Top25'],
      dtype='object')

In [ ]:
cols=['Top1', 'Top2', 'Top3', 'Top4', 'Top5', 'Top6', 'Top7',
       'Top8', 'Top9', 'Top10', 'Top11', 'Top12', 'Top13', 'Top14', 'Top15',
       'Top16', 'Top17', 'Top18', 'Top19', 'Top20', 'Top21', 'Top22', 'Top23',
       'Top24', 'Top25']
df["all"] = ''
for i in range(len(cols)):
  try:
    df["all"]=df["all"] +" "+df[cols[i]]
  except:
    print("not possible")

In [ ]:
df["all"].iloc[0]

' hindranc oper extract leak report scorecard hugh instant hit buoy blue jack get skate ice cold alex chao maracana build unit deplet leicest prevail elliott spoil everton s parti hungri spur sens rich pick gunner wide easi target derbi rais glass strupar s debut doubl southgat strike leed pay penalti hammer hand robson youth lesson saint parti like s 1999 wear wolv turn lamb stump mike catch testi gough s taunt langer escap hit 167 flintoff injuri pile woe england hunter threaten jospin new battl somm kohl s successor drawn scandal differ men women sara denver nurs turn solicitor diana s landmin crusad put tori panic yeltsin s resign caught opposit flat foot russian roulett sold recov titl'

In [ ]:
df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,all
0,2000 01 03,0,hindranc oper extract leak report,scorecard,hugh instant hit buoy blue,jack get skate ice cold alex,chao maracana build unit,deplet leicest prevail elliott spoil everton s...,hungri spur sens rich pick,gunner wide easi target,...,hunter threaten jospin new battl somm,kohl s successor drawn scandal,differ men women,sara denver nurs turn solicitor,diana s landmin crusad put tori panic,yeltsin s resign caught opposit flat foot,russian roulett,sold,recov titl,hindranc oper extract leak report scorecard h...
1,2000 01 04,0,scorecard,best lake scene,leader german sleaz inquiri,cheerio boyo,main recommend,cubi kill fee,cubi kill fee,cubi kill fee,...,time live,dear doctor,irish court halt ira man s extradit northern i...,burundi peac initi fade rebel reject mandela m...,pe point way forward ecb,campaign keep pressur nazi war crime suspect,jane ratcliff,yet thing would n t know without movi,millennium bug fail bite,scorecard best lake scene leader german sleaz...
2,2000 01 05,0,coventri caught counter flo,unit s rival road rio,thatcher issu defenc trial video,polic help smith lay law everton,tale trautmann bear two retel,england rack,pakistan retali call video walsh,cullinan continu cape monopoli,...,necaxa mexico,real madrid spain,raja casablanca morocco,corinthian brazil,toni s pet project,al nassr saudi arabia,ideal holm show,pinochet leav hospit test,use link,coventri caught counter flo unit s rival road...
3,2000 01 06,1,pilgrim know progress,thatcher face ban,mcilroy call irish fight spirit,leicest bin stadium blueprint,unit brace mexican wave,aunti back fashion even dress look bit tatti,shoaib appeal goe top,hussain hurt shambl lay blame earlier damag,...,bbc worst hit digit tv begin bite,much pay,christma glitch,upend tabl chop line score goal,scientif evid unreli defenc claim,fusco win judici review extradit case,rebel thwart russian advanc,blair order shake up fail nh,lesson law s hard heart,pilgrim know progress thatcher face ban mcilr...
4,2000 01 07,1,hitch horlock,beckham unit surviv,breast cancer screen,alan parker,guardian reader whinger,hollywood beyond,ash diamond,whinger formid minor,...,want chloe lunett,return cane complet agenda,sleepi hollow greeneland,blunkett outlin vision 11,embattl dobson attack play pay later livingston,doom dome,north south divid,aitken releas jail,gone aloft,hitch horlock beckham unit surviv breast canc...


In [ ]:
from gensim.models import Word2Vec
model_ted = Word2Vec(sentences=df["all"], window=5, min_count=5, workers=4, sg=0)


In [ ]:
model_ted

In [ ]:
def vector_append(lit):
    temp=[]
    current=[]
    for i in lit:
        try:
            current=model_ted.wv[i]
        except:
            pass
        else:
            for j in current:
                temp.append(j)
    return temp



In [ ]:
df["vectors"]=df.iloc[:,-1].apply(vector_append)
df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,all,vectors
0,2000 01 03,0,hindranc oper extract leak report,scorecard,hugh instant hit buoy blue,jack get skate ice cold alex,chao maracana build unit,deplet leicest prevail elliott spoil everton s...,hungri spur sens rich pick,gunner wide easi target,...,kohl s successor drawn scandal,differ men women,sara denver nurs turn solicitor,diana s landmin crusad put tori panic,yeltsin s resign caught opposit flat foot,russian roulett,sold,recov titl,hindranc oper extract leak report scorecard h...,"[-0.24420992, 0.64587104, -0.15451981, 0.48921..."
1,2000 01 04,0,scorecard,best lake scene,leader german sleaz inquiri,cheerio boyo,main recommend,cubi kill fee,cubi kill fee,cubi kill fee,...,dear doctor,irish court halt ira man s extradit northern i...,burundi peac initi fade rebel reject mandela m...,pe point way forward ecb,campaign keep pressur nazi war crime suspect,jane ratcliff,yet thing would n t know without movi,millennium bug fail bite,scorecard best lake scene leader german sleaz...,"[-0.24420992, 0.64587104, -0.15451981, 0.48921..."
2,2000 01 05,0,coventri caught counter flo,unit s rival road rio,thatcher issu defenc trial video,polic help smith lay law everton,tale trautmann bear two retel,england rack,pakistan retali call video walsh,cullinan continu cape monopoli,...,real madrid spain,raja casablanca morocco,corinthian brazil,toni s pet project,al nassr saudi arabia,ideal holm show,pinochet leav hospit test,use link,coventri caught counter flo unit s rival road...,"[-0.24420992, 0.64587104, -0.15451981, 0.48921..."
3,2000 01 06,1,pilgrim know progress,thatcher face ban,mcilroy call irish fight spirit,leicest bin stadium blueprint,unit brace mexican wave,aunti back fashion even dress look bit tatti,shoaib appeal goe top,hussain hurt shambl lay blame earlier damag,...,much pay,christma glitch,upend tabl chop line score goal,scientif evid unreli defenc claim,fusco win judici review extradit case,rebel thwart russian advanc,blair order shake up fail nh,lesson law s hard heart,pilgrim know progress thatcher face ban mcilr...,"[-0.24420992, 0.64587104, -0.15451981, 0.48921..."
4,2000 01 07,1,hitch horlock,beckham unit surviv,breast cancer screen,alan parker,guardian reader whinger,hollywood beyond,ash diamond,whinger formid minor,...,return cane complet agenda,sleepi hollow greeneland,blunkett outlin vision 11,embattl dobson attack play pay later livingston,doom dome,north south divid,aitken releas jail,gone aloft,hitch horlock beckham unit surviv breast canc...,"[-0.24420992, 0.64587104, -0.15451981, 0.48921..."


In [ ]:
df['length'] = df['vectors'].str.len()
df

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top19,Top20,Top21,Top22,Top23,Top24,Top25,all,vectors,length
0,2000 01 03,0,hindranc oper extract leak report,scorecard,hugh instant hit buoy blue,jack get skate ice cold alex,chao maracana build unit,deplet leicest prevail elliott spoil everton s...,hungri spur sens rich pick,gunner wide easi target,...,differ men women,sara denver nurs turn solicitor,diana s landmin crusad put tori panic,yeltsin s resign caught opposit flat foot,russian roulett,sold,recov titl,hindranc oper extract leak report scorecard h...,"[-0.24420992, 0.64587104, -0.15451981, 0.48921...",69800
1,2000 01 04,0,scorecard,best lake scene,leader german sleaz inquiri,cheerio boyo,main recommend,cubi kill fee,cubi kill fee,cubi kill fee,...,irish court halt ira man s extradit northern i...,burundi peac initi fade rebel reject mandela m...,pe point way forward ecb,campaign keep pressur nazi war crime suspect,jane ratcliff,yet thing would n t know without movi,millennium bug fail bite,scorecard best lake scene leader german sleaz...,"[-0.24420992, 0.64587104, -0.15451981, 0.48921...",56400
2,2000 01 05,0,coventri caught counter flo,unit s rival road rio,thatcher issu defenc trial video,polic help smith lay law everton,tale trautmann bear two retel,england rack,pakistan retali call video walsh,cullinan continu cape monopoli,...,raja casablanca morocco,corinthian brazil,toni s pet project,al nassr saudi arabia,ideal holm show,pinochet leav hospit test,use link,coventri caught counter flo unit s rival road...,"[-0.24420992, 0.64587104, -0.15451981, 0.48921...",59800
3,2000 01 06,1,pilgrim know progress,thatcher face ban,mcilroy call irish fight spirit,leicest bin stadium blueprint,unit brace mexican wave,aunti back fashion even dress look bit tatti,shoaib appeal goe top,hussain hurt shambl lay blame earlier damag,...,christma glitch,upend tabl chop line score goal,scientif evid unreli defenc claim,fusco win judici review extradit case,rebel thwart russian advanc,blair order shake up fail nh,lesson law s hard heart,pilgrim know progress thatcher face ban mcilr...,"[-0.24420992, 0.64587104, -0.15451981, 0.48921...",70000
4,2000 01 07,1,hitch horlock,beckham unit surviv,breast cancer screen,alan parker,guardian reader whinger,hollywood beyond,ash diamond,whinger formid minor,...,sleepi hollow greeneland,blunkett outlin vision 11,embattl dobson attack play pay later livingston,doom dome,north south divid,aitken releas jail,gone aloft,hitch horlock beckham unit surviv breast canc...,"[-0.24420992, 0.64587104, -0.15451981, 0.48921...",52400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4096,2016 06 27,0,barclay rb share suspend trade tank 8,pope say church ask forgiv gay past treatment,poland shock xenophob abus pole uk,second referendum cabinet agre,scotland welcom join eu merkel alli say,sterl dip friday s 31 year low amid brexit unc...,neg news south african presid allow state broa...,surg hate crime u k follow u k brexit vote,...,turkey sorri down russian jet,edward snowden lawyer vow new push pardon obama,brexit opinion poll reveal major n t want seco...,conserv mp leav campaign leav campaign n t pos...,economist predict uk recess weaken pound follo...,new eu superst plan franc germani creat europe...,pakistani cleric declar transgend marriag lega...,barclay rb share suspend trade tank 8 pope sa...,"[-0.24420992, 0.64587104, -0.15451981, 0.48921...",171600
4097,2016 06 28,1,2 500 scientist australia want save great barr...,person detail 112 000 french polic offic uploa...,amp p cut unit kingdom sovereign credit rate a...,huge helium deposit found africa,ceo south african state broadcast quit shortli...,brexit cost investor 2 trillion worst one day ...,hong kong democraci activist call return briti...,brexit iceland presid say uk join triangl non ...,...,37 000 yr old skull borneo reveal surpris scie...,palestinian stone western wall worship polic s...,jean claud juncker

In [ ]:
df['length'].mean()

131246.75451439727

In [ ]:
df['length'].median()

100100.0

In [ ]:
df['length'].max()

301100

In [ ]:
df['length'].min()

28100

In [ ]:
df['length'].std()

64192.266198857214

In [ ]:
df['length'].mode()

0    68900
dtype: int64

In [ ]:
len(df)

4098

In [ ]:
df=df[df['length'] <= 164292]
df=df[df['length'] >= 35908]
df

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top19,Top20,Top21,Top22,Top23,Top24,Top25,all,vectors,length
0,2000 01 03,0,hindranc oper extract leak report,scorecard,hugh instant hit buoy blue,jack get skate ice cold alex,chao maracana build unit,deplet leicest prevail elliott spoil everton s...,hungri spur sens rich pick,gunner wide easi target,...,differ men women,sara denver nurs turn solicitor,diana s landmin crusad put tori panic,yeltsin s resign caught opposit flat foot,russian roulett,sold,recov titl,hindranc oper extract leak report scorecard h...,"[-0.24420992, 0.64587104, -0.15451981, 0.48921...",69800
1,2000 01 04,0,scorecard,best lake scene,leader german sleaz inquiri,cheerio boyo,main recommend,cubi kill fee,cubi kill fee,cubi kill fee,...,irish court halt ira man s extradit northern i...,burundi peac initi fade rebel reject mandela m...,pe point way forward ecb,campaign keep pressur nazi war crime suspect,jane ratcliff,yet thing would n t know without movi,millennium bug fail bite,scorecard best lake scene leader german sleaz...,"[-0.24420992, 0.64587104, -0.15451981, 0.48921...",56400
2,2000 01 05,0,coventri caught counter flo,unit s rival road rio,thatcher issu defenc trial video,polic help smith lay law everton,tale trautmann bear two retel,england rack,pakistan retali call video walsh,cullinan continu cape monopoli,...,raja casablanca morocco,corinthian brazil,toni s pet project,al nassr saudi arabia,ideal holm show,pinochet leav hospit test,use link,coventri caught counter flo unit s rival road...,"[-0.24420992, 0.64587104, -0.15451981, 0.48921...",59800
3,2000 01 06,1,pilgrim know progress,thatcher face ban,mcilroy call irish fight spirit,leicest bin stadium blueprint,unit brace mexican wave,aunti back fashion even dress look bit tatti,shoaib appeal goe top,hussain hurt shambl lay blame earlier damag,...,christma glitch,upend tabl chop line score goal,scientif evid unreli defenc claim,fusco win judici review extradit case,rebel thwart russian advanc,blair order shake up fail nh,lesson law s hard heart,pilgrim know progress thatcher face ban mcilr...,"[-0.24420992, 0.64587104, -0.15451981, 0.48921...",70000
4,2000 01 07,1,hitch horlock,beckham unit surviv,breast cancer screen,alan parker,guardian reader whinger,hollywood beyond,ash diamond,whinger formid minor,...,sleepi hollow greeneland,blunkett outlin vision 11,embattl dobson attack play pay later livingston,doom dome,north south divid,aitken releas jail,gone aloft,hitch horlock beckham unit surviv breast canc...,"[-0.24420992, 0.64587104, -0.15451981, 0.48921...",52400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3993,2016 01 29,1,13 year old girl russian speak commun berlin k...,two jail hand isi leaflet london s oxford street,mob hundr mask men rampag stockholm central st...,russia allow exclus kurd syrian peac talk,india egypt say thank free internet facebook,snow fall kuwait first time ever,mexican cop center mass disappear,church england member back same sex marriag,...,spain princess lose legal battl avoid tax frau...,anti gmo research may base manipul data,fukushima nuclear crisi far kan say japan time,uk mother tareena shakil took son syria guilti...,chines billionair zhang xin the old model n t ...,russian fighter came within 15 feet u s air fo...,egypt minist justic call kill 400 000 muslim b...,13 year old girl russian speak commun berlin ...,"[-0.24420992, 0.64587104, -0.15451981, 0.48921...",157700
4001,2016 02 10,0,british isi fighter call superman return uk sy...,anim reduc cannib egyptian zoo neglect,grow thicker skin germani tell poland german g...,least 50 kill prison riot monterrey mexico,north korea accus south korea danger declar wa...,cathol bishop oblig report cleric child abus v...,british child blow isi prison new execut video,saudi arabia may deploy troop syria counter ru...,...,master forger arrest thailand fake passport mi...,scandinavia first women onli mosqu women imam ...

In [ ]:
len(df)

2411

In [ ]:
print("test")

test


In [ ]:
def padding(vec):
    rn=100100-len(vec)
    for i in range(rn):
        vec.append(0)
    return vec

In [ ]:
df['padded_vectors']=df['vectors'].apply(padding)

In [ ]:
df['padded_length'] = df['padded_vectors'].str.len()
df

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top21,Top22,Top23,Top24,Top25,all,vectors,length,padded_vectors,padded_length
0,2000 01 03,0,hindranc oper extract leak report,scorecard,hugh instant hit buoy blue,jack get skate ice cold alex,chao maracana build unit,deplet leicest prevail elliott spoil everton s...,hungri spur sens rich pick,gunner wide easi target,...,diana s landmin crusad put tori panic,yeltsin s resign caught opposit flat foot,russian roulett,sold,recov titl,hindranc oper extract leak report scorecard h...,"[-0.24420992, 0.64587104, -0.15451981, 0.48921...",69800,"[-0.24420992, 0.64587104, -0.15451981, 0.48921...",100100
1,2000 01 04,0,scorecard,best lake scene,leader german sleaz inquiri,cheerio boyo,main recommend,cubi kill fee,cubi kill fee,cubi kill fee,...,pe point way forward ecb,campaign keep pressur nazi war crime suspect,jane ratcliff,yet thing would n t know without movi,millennium bug fail bite,scorecard best lake scene leader german sleaz...,"[-0.24420992, 0.64587104, -0.15451981, 0.48921...",56400,"[-0.24420992, 0.64587104, -0.15451981, 0.48921...",100100
2,2000 01 05,0,coventri caught counter flo,unit s rival road rio,thatcher issu defenc trial video,polic help smith lay law everton,tale trautmann bear two retel,england rack,pakistan retali call video walsh,cullinan continu cape monopoli,...,toni s pet project,al nassr saudi arabia,ideal holm show,pinochet leav hospit test,use link,coventri caught counter flo unit s rival road...,"[-0.24420992, 0.64587104, -0.15451981, 0.48921...",59800,"[-0.24420992, 0.64587104, -0.15451981, 0.48921...",100100
3,2000 01 06,1,pilgrim know progress,thatcher face ban,mcilroy call irish fight spirit,leicest bin stadium blueprint,unit brace mexican wave,aunti back fashion even dress look bit tatti,shoaib appeal goe top,hussain hurt shambl lay blame earlier damag,...,scientif evid unreli defenc claim,fusco win judici review extradit case,rebel thwart russian advanc,blair order shake up fail nh,lesson law s hard heart,pilgrim know progress thatcher face ban mcilr...,"[-0.24420992, 0.64587104, -0.15451981, 0.48921...",70000,"[-0.24420992, 0.64587104, -0.15451981, 0.48921...",100100
4,2000 01 07,1,hitch horlock,beckham unit surviv,breast cancer screen,alan parker,guardian reader whinger,hollywood beyond,ash diamond,whinger formid minor,...,embattl dobson attack play pay later livingston,doom dome,north south divid,aitken releas jail,gone aloft,hitch horlock beckham unit surviv breast canc...,"[-0.24420992, 0.64587104, -0.15451981, 0.48921...",52400,"[-0.24420992, 0.64587104, -0.15451981, 0.48921...",100100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3993,2016 01 29,1,13 year old girl russian speak commun berlin k...,two jail hand isi leaflet london s oxford street,mob hundr mask men rampag stockholm central st...,russia allow exclus kurd syrian peac talk,india egypt say thank free internet facebook,snow fall kuwait first time ever,mexican cop center mass disappear,church england member back same sex marriag,...,fukushima nuclear crisi far kan say japan time,uk mother tareena shakil took son syria guilti...,chines billionair zhang xin the old model n t ...,russian fighter came within 15 feet u s air fo...,egypt minist justic call kill 400 000 muslim b...,13 year old girl russian speak commun berlin ...,"[-0.24420992, 0.64587104, -0.15451981, 0.48921...",157700,"[-0.24420992, 0.64587104, -0.15451981, 0.48921...",157700
4001,2016 02 10,0,british isi fighter call superman return uk sy...,anim reduc cannib egyptian zoo neglect,grow thicker skin germani tell poland german g...,least 50 kill prison riot monterrey mexico,north korea accus south korea danger declar wa...,cathol bishop oblig report cleric child abus v...,british child blow isi prison new execut video,saudi arabia may deploy troop syria counter ru...,...,turkey s leader blame u s sea blood region,n korea start feel neighbor wrath rocket launch,pope franci despa

In [ ]:
# df.to_csv("vectorised_sentiment_padded.csv")  

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report

In [ ]:
X = df['padded_vectors'].to_numpy()
X = X.reshape(-1, 1)

In [ ]:
X.shape

(2411, 1)

In [ ]:
X[0]

In [ ]:
import numpy as np

In [ ]:
X = np.concatenate(np.concatenate(X, axis = 0), axis = 0).reshape(2411, 106481)

ValueError: cannot reshape array of size 256725100 into shape (2411,106481)

In [ ]:
X.shape

(2411, 1)

In [ ]:
y = df['Label']

In [ ]:
y.shape

(2411,)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)

In [ ]:
X_train.shape, X_test.shape

((1928, 1), (483, 1))

In [ ]:
clf = LogisticRegression(solver='liblinear')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

ValueError: setting an array element with a sequence.